In [5]:
import cv2
import time
from ultralytics import YOLO
import cvzone
import numpy as np
# import serial
# ser = serial.Serial('COM6', 9600)  # Adjust COM port as necessary
# ser1 = serial.Serial('COM7', 9600)  # Adjust COM port as necessary
# time.sleep(2)

In [6]:
# Load a custom model
model = YOLO('best.pt')

ClassNames = ["Cadbury_Chocolate","Kitkat_Chocolate","Mars_Chocolate","Snickers_Chocolate"]

# Confidence threshold for object detection
confidence_threshold = 0.7

# Used to read frames from a video stream
capture_frame = cv2.VideoCapture(0)

In [7]:
# fuction to get coordinates of the frame
def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE:
        axis = [x, y]
        print(axis)

In [8]:
while True:
    #Read the Capture Frame if sucessfull ret is true
    ret, frame = capture_frame.read()

    if not ret:
        print("\n Failed to grab frame")
        break

    # Resize according to the Model Req.
    frame = cv2.resize(frame, (640, 480))

    # Frame Flip to Easly View
    frame = cv2.flip(frame, 1)

    # Start time capture
    start_time = time.time()

    # Perform object detection on the flipped frame
    results = model(frame)[0]

    # End time capture
    end_time = time.time()

    # Calculate processing time
    elapsed_time = end_time - start_time

    # Calculate Frames per Second
    fps = 1 / elapsed_time

    # Print FPS on Screen
    cv2.putText(frame, "FPS : {:.2f}".format(fps), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    #Limit line coordinates
    start_point = (0, 262)
    end_point = (667, 262)

    #Limit Line at which the conveyor belt Stop
    cv2.line(frame, start_point, end_point, (0, 255, 0), thickness=1)

    #Results Edit to make configuration according to logic
    for r in results:
        boxes = r.boxes
        for box in boxes:
            #Bounding Boxes
            x1,y1,x2,y2= box.xyxy[0]
            x1, y1, x2, y2 = int(x1),int(y1),int(x2),int(y2)
            w, h = x2 - x1, y2 - y1             #h means height , w means width
            #Confidence
            conf = box.conf[0]
            # Class Name
            cls = int (box.cls[0])

            if conf > confidence_threshold:
                if y1 <= start_point[1] and y2 >= start_point[1]:
                    cvzone.cornerRect(frame, (x1, y1, w, h))
                    cvzone.putTextRect(frame, f'#{cls} {ClassNames[cls]} ({conf:.2f})', (max(0, x1), max(35, y1)), 1, 1,
                                       (255, 255, 255), (255, 0, 255))
                    #ser1.write(b'Stop')          #for stop condition to the conveyor belt
                    # time.sleep(2)

                    #Class logic for the sending data to Arduino
                    if (cls == 0):
                        # ser.write(b'A')
                        print("cadbury detected : index", cls)
                        # time.sleep(2)

                    if (cls == 1):
                        # ser.write(b'B')
                        print("kitkat detected : index", cls)
                        # time.sleep(2)

                    if (cls == 2):
                        # ser.write(b'C')
                        print("Mars detected : index", cls)
                        # time.sleep(2)

                    if (cls == 3):
                        # ser.write(b'D')
                        print("snickers detected : index", cls)
                        # time.sleep(2)

    #Visulize the Results
    cv2.imshow('Object Detection', frame)

    # use this command to get the coordinate of object detectd screen
    cv2.setMouseCallback('Object Detection', RGB)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) == ord('q'):
        break

# Release the capture
capture_frame.release()
cv2.destroyAllWindows()


0: 480x640 1 cadbury_chocolate, 700.1ms
Speed: 34.2ms preprocess, 700.1ms inference, 2427.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 cadbury_chocolates, 684.9ms
Speed: 9.0ms preprocess, 684.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 cadbury_chocolate, 606.6ms
Speed: 5.1ms preprocess, 606.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 755.3ms
Speed: 4.0ms preprocess, 755.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 659.6ms
Speed: 3.6ms preprocess, 659.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 766.2ms
Speed: 5.0ms preprocess, 766.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 736.9ms
Speed: 4.1ms preprocess, 736.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 cadbury_chocolate, 651.8